# Compute Mesh Surface Area

## Connect to the database server

In [ ]:
 # DatatJoint configuration and connection

import datajoint as dj
import ipyvolume.pylab as p3
import numpy as np
from funconnect import nda, ta3
from funconnect.connectomics import MeshSurfaceArea

## Combined structural and functional pipeline

In [ ]:
dj.ERD(ta3) + dj.ERD(nda)

## Mesh Surface Area

In [ ]:
### define table to compute surface area of a Mesh Fragment
@schema
class MeshSurfaceArea(dj.Computed):
    definition = """
    # computes total surface area of a mesh with soma
    -> ta3.Mesh
    ---
    surface_area: float         # surface area of a mesh fragment in um^2
    """
    
    key_source = ta3.Mesh & ta3.Soma
    
    def make(self, key):
        vertices, triangles = (ta3.Mesh.Fragment & key).fetch('vertices', 'triangles')
        vertices = vertices / 1000 # change units from nm to um
        frag_area_list = []
        # computing area for each fragment of mesh
        for vert, triang in zip(vertices, triangles):
            tri_area_list = []
            for tri_xyz in vert[triang]:
                # computing area of triangle
                x = np.cross(tri_xyz[0]-tri_xyz[1], tri_xyz[0]-tri_xyz[2])
                area_triang = np.linalg.norm(x) / 2
                tri_area_list.append(area_triang)
            frag_area = np.sum(tri_area_list)
            frag_area_list.append(frag_area)
        mesh_area = np.sum(frag_area_list)
        key['surface_area'] = mesh_area
        self.insert1(key)
        print('Computed surface area for segment_id {segment_id}'.format(**key))

In [ ]:
MeshSurfaceArea()